In [1]:
from math import *
from numpy import *
from scipy import linalg
T = 10
L = 20 # nombre de pas
t = linspace(0,T,L)
# print(t)
dt = t[2]-t[1]
print("dt=" +str(dt))
H0 = array([[1, 0], [0, 2]])
H1 = array([[0, 1], [1, 0]])
y0 = [1, 0]
y0 = transpose(y0)
ycible = [0, 1]
ycible = transpose(ycible)
c = ones((1,L-1))
alpha = 10
maxiter = 30
shape(c)

dt=0.5263157894736842


(1, 19)

Calculons y

In [2]:
def splitting (H0, H1, dt, L, c, y0):
    y = zeros((2,L))
    y[:,0] = y0
#     a=shape(linalg.expm((-1j)*H0*dt))
#     b=shape(linalg.expm((-1j)*H1*c[:,L-2]*dt))
#     d=shape(linalg.expm((-1j)*H1*c[:,L-2]*dt)@y[:,L-1])
#     print("a "+str(a)+";b= "+str(b)+";d "+str(d))
    for i in range(1,L):
#         print(i)
        y[:,i] = linalg.expm((-1j)*H0*dt)@linalg.expm((-1j)*H1*c[:,i-1]*dt)@y[:,i-1]
    return y

In [3]:
y = splitting(H0, H1, dt, L, c, y0)
print(shape(y))

(2, 20)


/home/kenn/Documents/python/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


Calculons J

In [4]:
def fonctionnelle (H0, H1, dt, L, c, ycible, y):
    alpha = 10
    yL=y[:,L-1]
    c = c**2 # on eleve chaque terme de c au carree
    c=sum(c) # somme des termes de c
    return 1/2*linalg.norm(yL-ycible)**2 + alpha * dt * c

In [5]:
J = fonctionnelle(H0, H1, dt, L, c, ycible, y)
print(J)

100.83210297268938


Calculons p

In [6]:
def calculP (H0, H1, dt, L, c, ycible, y):
    p = zeros((2,L))
    yL = y[:,L-1]
#     print(yL-ycible)
#     print(p[:,L-1])
    p[:,L-1] = yL-ycible # p(T) de l'equation (3), yL=y(T)
    for i in range(L-2,-1,-1): # backward range(start,end+1,pas)
        p[:,i] = linalg.expm((-1j)*H1*c[:,i]*dt)@linalg.expm((-1j)*H0*dt)@p[:,i+1]
    return p

# for i in range(20,-1,-1):
#     print(i, end=" ")

In [7]:
p = calculP (H0, H1, dt, L, c, ycible, y)
print(shape(p))

(2, 20)


/home/kenn/Documents/python/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


Calculons gradJ

In [8]:
def gradJ (H0, H1, dt, L, c, ycible, y):
    alpha = 10
    p = calculP(H0, H1, dt, L, c, ycible, y)
    p = transpose(p)
    grad = 2*alpha*dt*c
    for i in range(0,L-1):
        grad[:,i] = grad[:,i] + real(p[i+1:i+2:,]@linalg.expm((-1j)*H0*dt)@((-1j)*H1*dt)@linalg.expm((-1j)*H1*c[:,i]*dt)@y[:,i])
    return grad

In [9]:
grad = gradJ (H0, H1, dt, L, c, ycible, y)
print(shape(grad))

(1, 19)


/home/kenn/Documents/python/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


Debug gradient: On verifie que le taux de variation - gradJ tend bien vers 0

In [10]:
def debuggrad (H0, H1, dt, L, c, ycible, y):
    c = random.rand(1,L-1)
    dc = random.rand(1,L-1)
    grad = gradJ (H0, H1, dt, L, c, ycible, y)
    for i in range(1,7):
        epsi = 10**(-i)
        Jp = fonctionnelle (H0, H1, dt, L, c+epsi*dc, ycible, y)
        Jm = fonctionnelle (H0, H1, dt, L, c-epsi*dc, ycible, y)
        conv = (Jp-Jm)/(2*epsi)-grad@(transpose(dc))
        print(conv)
    

In [11]:
debuggrad (H0, H1, dt, L, c, ycible, y)

[[-0.37247366]]
[[-0.37247366]]
[[-0.37247366]]
[[-0.37247366]]
[[-0.37247366]]
[[-0.37247366]]


/home/kenn/Documents/python/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


Calculons enfin c

In [12]:
def controle(H0, H1, dt, L, c, ycible, y):
    rho = 0.1
    for i in range(1,maxiter):
        J = fonctionnelle(H0, H1, dt, L, c, ycible, y)
        grad = gradJ(H0, H1, dt, L, c, ycible, y)
        c = c - rho*grad
        print(linalg.norm(grad))

In [13]:
c = controle(H0, H1, dt, L, c, ycible, y)

/home/kenn/Documents/python/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


46.024483139148096
2.4465156190935784
0.13627080265659694
0.007568322412326191
0.000418966670375718
2.3121260623292747e-05
1.2722808834011928e-06
6.982176665707253e-08
3.8223570380375314e-09
2.0878303539019493e-10
1.1380614931465164e-11
6.191970906571704e-13
3.362216142152035e-14
1.811802116572607e-15
1.0082632301392273e-16
1.5418044880947972e-17
1.3911683624122383e-17
1.3884562595870138e-17
1.3884562419453365e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
1.3884562418242266e-17
